<a href="https://colab.research.google.com/github/nakshathranazer/QuantumMul/blob/main/QuantumMultiplier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
%pip install qiskit
%pip install pylatexenc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
from qiskit import *

In [53]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def executeQFT(qc, reg, n, pie):
    # Computes the quantum Fourier transform of reg, one qubit a time
    # Apply one Hadamard gate to the n-th qubit of the quantum register reg, and 
    # then apply repeated phase rotations with parameters being pi divided by 
    # increasing powers of two
    
    qc.h(reg[n])    
    for i in range(0, n):
        qc.cp(pie/float(2**(i+1)), reg[n-(i+1)], reg[n])    

def inverseQFT(qc, reg, n, pie):
    # Performs the inverse quantum Fourier transform on a register reg.
    # Apply repeated phase rotations with parameters being pi divided by 
    # decreasing powers of two, and then apply a Hadamard gate to the nth qubit
    # of the register reg.
    
    for i in range(n):
        qc.cp(-1*pie/float(2**(n-i)), reg[i], reg[n])
    qc.h(reg[n])
    
def initQubits(str, qc, reg, n):
    # Flip the corresponding qubit in register if a bit in the string is a 1
    for i in range(n):
        if str[i] == "1":
            qc.x(reg[n-(i+1)])
def binaryTodecimal(n):
    decimal = 0
    power = 1
    while n>0:
        rem = n%10
        n = n//10
        decimal += rem*power
        power = power*2
        
    return decimal
def printResult(first, second, qc,result, cl, n, operator):

    # Measure qubits
    for i in range(n+1):
        qc.measure(result[i], cl[i])

    # Execute using the local simulator
    
    # Set chosen backend and execute job
    num_shots = 100 #Setting the number of times to repeat measurement

    job = execute(qc, backend=Aer.get_backend('qasm_simulator'), shots=num_shots)
    # Get results of program
    job_stats = job.result().get_counts()
    for key, value in job_stats.items():
        res = key
        prob = value

    
    print(binaryTodecimal(int(res)))

In [54]:
import math
pie = math.pi

def sum(a, b, qc):
    
    n = len(a)-1
    # Compute the Fourier transform of register a
    for i in range(n+1):
        executeQFT(qc, a, n-i, pie)
        
    # Add the two numbers by evolving the Fourier transform F(ψ(reg_a))>
    # to |F(ψ(reg_a+reg_b))>
    for i in range(n+1):
        evolveQFTStateSum(qc, a, b, n-i, pie) 
        
    # Compute the inverse Fourier transform of register a
    for i in range(n+1):
        inverseQFT(qc, a, i, pie)


def evolveQFTStateSum(qc, reg_a, reg_b, n, pie):
    # Evolves the state |F(ψ(reg_a))> to |F(ψ(reg_a+reg_b))> using the quantum 
    # Fourier transform conditioned on the qubits of the reg_b.
    # Apply repeated phase rotations with parameters being pi divided by 
    # increasing powers of two.
    l = len(reg_b)
    for i in range(n+1):
        if (n - i) > l - 1:
            pass
        else:
            qc.cp(pie/float(2**(i)), reg_b[n-i], reg_a[n])        

In [55]:
def multiply(a, secondDec, result, qc):
    
    n = len(a) -1
    # Compute the Fourier transform of register 'result'
    for i in range(n+1):
        executeQFT(qc, result, n-i, pie)
    
    # Add the two numbers by evolving the Fourier transform F(ψ(reg_a))>
    # to |F(ψ((second * reg_a))>, where we loop on the sum as many times as 'second' says, 
    # doing incremental sums
    for j in range(secondDec):
        for i in range(n+1):
            evolveQFTStateSum(qc, result, a, n-i, pie)

    # Compute the inverse Fourier transform of register a
    for i in range(n+1):
        inverseQFT(qc, result, i, pie)

In [56]:
def multiplier(x,y):
    


  # check the inputs
  while (x < 0 or x > 2047) or (y< 0 or y > 2047):
    if x < 0 or x > 2047:
        print(bcolors.FAIL + "Invalid first input number" + bcolors.ENDC)
        

    if y < 0 or y > 2047:
        print(bcolors.FAIL + "Invalid second input number" + bcolors.ENDC)
       


  first = '{0:{fill}3b}'.format(x, fill='0')
  second = '{0:{fill}3b}'.format(y, fill='0')
  # for multiplication
  firstDec = x
  secondDec = y


  l1 = len(first)
  l2 = len(second)

  # Padding 'first' the same lenght of 'result'
  # since result can have at max len(first) + len(second) bits when multiplying
  first = ("0")*(l2) + first
  n = l1+l2


  # create the register based on the operation choosen
  a = QuantumRegister(n+1, "a") 
  b = QuantumRegister(n+1, "b")
  accumulator = QuantumRegister(n+1, "accumulator")     
  cl = ClassicalRegister(n+1, "cl")


  
  qc = QuantumCircuit(a, b, cl, name="qc")
  # Flip the corresponding qubit in register a if a bit in the string first is a 1
  initQubits(first, qc, a, n)

  multiply(a,secondDec,b,qc)
  operator = '*'
  printResult(first, second, qc, b, cl, n, operator)
  


In [61]:
multiplier(6,5)

30
